In [5]:
from manim import *
import numpy as np

class StrategicNetwork(Scene):
    def construct(self):
        # ---------------------------------------------------------
        # 1. SETUP THE DATA
        # ---------------------------------------------------------
        # We build the graph structure manually to ensure the "Hub and Spoke" feel
        # Node 0 is the center.
        vertices = [0]
        edges = []
        
        # Layer 1: 5 nodes connected to center
        layer_1 = [1, 2, 3, 4, 5]
        vertices.extend(layer_1)
        for n in layer_1:
            edges.append((0, n))
            
        # Layer 2: ~3 nodes for each Layer 1 node
        layer_2 = []
        current_id = 6
        for parent in layer_1:
            # We add 3 children to each
            for _ in range(3):
                vertices.append(current_id)
                edges.append((parent, current_id))
                layer_2.append(current_id)
                current_id += 1
                
        # Layer 3: A few sporadic nodes off layer 2 (to look "messy" and distant)
        layer_3 = []
        for parent in layer_2[:8]: # Just add to the first few for asymmetry
            vertices.append(current_id)
            edges.append((parent, current_id))
            layer_3.append(current_id)
            current_id += 1

        # We define our "Winner" path (Root -> L1 -> L2) beforehand
        # Let's say the path is 0 -> 2 -> 8
        best_node = 8
        path_nodes = [0, 2, 8]
        path_edges = [(0, 2), (2, 8)]

        # ---------------------------------------------------------
        # 2. DRAW THE GRAPH
        # ---------------------------------------------------------
        
        # We fix node 0 at the center so the spring layout revolves around it
        layout_config = {"fixed": {0: ORIGIN}}
        
        g = Graph(
            vertices, 
            edges, 
            layout="spring", 
            layout_scale=3.5,
            layout_config=layout_config,
            vertex_config={
                0: {"color": GREEN, "radius": 0.15}, # Special root config
            },
            labels=False 
        )
        
        # Set default style for non-root nodes
        for v in vertices:
            if v != 0:
                g.vertices[v].set_color(GREY)
                g.vertices[v].scale(0.8)

        self.play(Create(g), run_time=2)
        self.wait()

        # ---------------------------------------------------------
        # 3. DYNAMIC BUDGET CUTOFF
        # ---------------------------------------------------------
        
        # Draw the cutoff circle
        cutoff_radius = 2.8
        cutoff_circle = Circle(radius=cutoff_radius, color=RED, fill_opacity=0.1)
        cutoff_label = Text("Dynamic Budget Cutoff", font_size=20, color=RED).next_to(cutoff_circle, UP)

        self.play(Create(cutoff_circle), Write(cutoff_label))
        
        # Identify nodes outside the circle
        nodes_outside = []
        for v in vertices:
            # Get the mobject's coordinates
            pos = g.vertices[v].get_center()
            # Check distance from origin
            if np.linalg.norm(pos) > cutoff_radius:
                nodes_outside.append(v)

        # "Cut out" the farther nodes (fade them and their edges)
        anims = []
        for v in nodes_outside:
            anims.append(g.vertices[v].animate.set_opacity(0.2))
            # Find edges connected to this node and fade them
            for e in edges:
                if v in e:
                    if e in g.edges:
                        anims.append(g.edges[e].animate.set_opacity(0.2))
        
        self.play(*anims, run_time=1.5)
        self.wait()

        # ---------------------------------------------------------
        # 4. STRATEGIC CENTERS (Identification)
        # ---------------------------------------------------------
        
        # We pick a few "strategic" nodes to highlight inside the circle
        # Including our "best_node" (8) and maybe two others like 6 and 10
        strategic_nodes = [6, 10, best_node]
        
        s_anims = []
        for n in strategic_nodes:
            # Make them Blue and larger
            s_anims.append(g.vertices[n].animate.set_color(BLUE).scale(1.5))
            
        # Make the "Best" one even bigger and deeper blue
        s_anims.append(g.vertices[best_node].animate.set_color(DARK_BLUE).scale(1.8))
        
        centers_label = Text("Strategic Centers Identified", font_size=24, color=BLUE)
        centers_label.to_corner(UL)

        self.play(*s_anims, Write(centers_label))
        self.wait()

        # ---------------------------------------------------------
        # 5. DRAW PATH TO BEST CENTER
        # ---------------------------------------------------------
        
        path_anims = []
        # Trace path 0 -> 2 -> 8
        # Color edges Yellow
        for edge in path_edges:
            path_anims.append(g.edges[edge].animate.set_color(YELLOW).set_stroke(width=6))
        
        # Color nodes Yellow (except start node 0, keep that Green)
        for node in path_nodes:
            if node != 0:
                path_anims.append(g.vertices[node].animate.set_color(YELLOW))

        self.play(*path_anims, run_time=1.5)
        self.wait()

        # ---------------------------------------------------------
        # 6. OPTIMAL STEP (The First Move)
        # ---------------------------------------------------------
        
        # The first step is Node 2. 
        # We want to circle the edge (0,2) or the node 2? 
        # User said: "circle the first step... color it green"
        
        first_step_node = path_nodes[1] # Node 2
        
        # Create a circle around the first node
        selection_circle = Circle(radius=0.3, color=GREEN).move_to(g.vertices[first_step_node])
        
        # Label it
        step_label = Text("Optimal Step", font_size=20, color=GREEN)
        step_label.next_to(selection_circle, DOWN)
        
        self.play(
            Create(selection_circle),
            g.vertices[first_step_node].animate.set_color(GREEN),
            Write(step_label)
        )
        
        self.wait(2)

/Users/willott/School/RESEARCH/IonQ/IonQ2/2026-IonQ/venv/lib/python3.14/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
